In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet152V2 import ResNet152V2
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
#from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [224, 224]

train_path = '/content/gdrive/MyDrive/Apple/train'
valid_path = '/content/gdrive/MyDrive/Apple/train'

In [6]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [7]:
for layer in inception.layers:
    layer.trainable = False

In [8]:
folders = glob('/content/gdrive/MyDrive/Apple/train/*')

In [9]:
x = Flatten()(inception.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Apple/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2497 images belonging to 4 classes.


In [15]:
training_set.class_indices

{'Apple_scab': 0, 'Black_rot': 1, 'apple_rust': 2, 'healthy': 3}

In [18]:
print(training_set.class_indices) #prints every single key and class of that dataset


{'Apple_scab': 0, 'Black_rot': 1, 'apple_rust': 2, 'healthy': 3}


In [19]:
labels = '\n'.join(sorted(training_set.class_indices.keys())) #print all these keys as a list of labels into a text file called labels.txt
with open('labels.txt', 'w') as f: #writes to the labels.txt file, and if it doesnt exists, it creates one, and if it does exist, it will overrite it. (thats what 'w' is for)
    f.write(labels)

In [ ]:
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/Apple/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 731 images belonging to 4 classes.


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


KeyboardInterrupt: ignored

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='training loss')
plt.plot(r.history['val_loss'], label='validation loss')
plt.legend()
plt.show()
plt.savefig('LossValidation_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='training accuracy')
plt.plot(r.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show()
plt.savefig('AccuracyValidation_acc')

NameError: ignored

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_inception.h5')

In [ ]:
y_pred = model.predict(test_set)

KeyboardInterrupt: ignored

In [ ]:
y_pred

array([[4.0258192e-14, 9.9969208e-01, 4.1490189e-22, 3.0794353e-04],
       [1.1309654e-19, 1.0623769e-30, 1.0000000e+00, 0.0000000e+00],
       [2.3693729e-22, 1.0000000e+00, 3.7505146e-35, 4.7006916e-24],
       ...,
       [1.7283222e-12, 2.0405485e-30, 1.0000000e+00, 1.6491989e-23],
       [2.1581355e-31, 1.0000000e+00, 0.0000000e+00, 1.2309601e-37],
       [1.3606085e-26, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00]],
      dtype=float32)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

array([1, 2, 1, 1, 0, 3, 0, 0, 3, 3, 3, 3, 3, 1, 3, 3, 0, 2, 2, 1, 3, 3,
       3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 1, 3, 1, 0, 3, 2, 1, 3, 3, 3, 3, 0,
       0, 3, 1, 3, 3, 2, 1, 1, 0, 2, 2, 3, 3, 1, 3, 3, 0, 3, 1, 1, 2, 0,
       0, 3, 2, 1, 3, 1, 3, 0, 3, 0, 3, 0, 0, 1, 3, 2, 2, 0, 3, 0, 0, 3,
       3, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 2, 2, 3, 3, 3,
       0, 1, 3, 3, 0, 0, 3, 3, 2, 3, 0, 1, 2, 3, 1, 0, 3, 3, 3, 1, 3, 3,
       3, 1, 0, 1, 0, 3, 0, 1, 3, 0, 2, 3, 2, 2, 2, 2, 3, 1, 3, 1, 2, 3,
       2, 2, 1, 2, 3, 0, 0, 3, 2, 1, 3, 0, 0, 1, 3, 0, 3, 3, 0, 3, 3, 3,
       3, 3, 3, 2, 1, 1, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3,
       2, 3, 3, 3, 1, 3, 3, 3, 0, 3, 3, 3, 0, 3, 1, 2, 3, 2, 2, 1, 0, 2,
       1, 2, 1, 1, 0, 0, 1, 0, 3, 0, 3, 3, 1, 3, 0, 3, 2, 0, 3, 3, 3, 2,
       2, 0, 3, 1, 3, 3, 0, 3, 3, 0, 1, 1, 3, 3, 2, 3, 3, 0, 2, 3, 1, 1,
       2, 3, 3, 3, 3, 2, 3, 3, 1, 1, 0, 3, 2, 2, 2, 3, 3, 0, 3, 3, 0, 2,
       3, 3, 2, 3, 3, 0, 3, 1, 1, 3, 1, 0, 0, 1, 0,

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_inception.h5')

In [ ]:
img=image.load_img('/content/gdrive/MyDrive/Apple/test/apple_rust/image (172).JPG',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[112., 103.,  94.],
        [112., 103.,  94.],
        [112., 103.,  94.],
        ...,
        [138., 132., 134.],
        [137., 131., 133.],
        [136., 130., 132.]],

       [[112., 103.,  94.],
        [112., 103.,  94.],
        [112., 103.,  94.],
        ...,
        [136., 130., 132.],
        [135., 129., 131.],
        [135., 129., 131.]],

       [[113., 104.,  95.],
        [112., 103.,  94.],
        [112., 103.,  94.],
        ...,
        [134., 128., 130.],
        [133., 127., 129.],
        [133., 127., 129.]],

       ...,

       [[127., 121., 121.],
        [128., 122., 122.],
        [127., 121., 121.],
        ...,
        [119., 113., 113.],
        [114., 108., 108.],
        [107., 101., 101.]],

       [[132., 126., 126.],
        [133., 127., 127.],
        [130., 124., 124.],
        ...,
        [114., 108., 108.],
        [113., 107., 107.],
        [110., 104., 104.]],

       [[140., 134., 134.],
        [137., 131., 131.],
        [127., 1

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[9.9926406e-01, 4.5585486e-07, 6.9470808e-04, 4.0790419e-05]],
      dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a

array([0])

In [ ]:
a==1

array([False])